<a href="https://colab.research.google.com/github/Ishita2502/ImagePred/blob/main/Logistic_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
from google.colab import drive
drive.mount('/gdrive')


Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [ ]:
path = '/gdrive/My Drive/Online MLCC : Oct 2021/Data/Classwork/'

In [ ]:
data = pd.read_csv(path+'paint_mnist.csv', header=None)
data.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,745,746,747,748,749,750,751,752,753,754,755,756,757,758,759,760,761,762,763,764,765,766,767,768,769,770,771,772,773,774,775,776,777,778,779,780,781,782,783,784
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,255,255,255,255,255,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
data.shape

(3599, 785)

In [ ]:
x=data.drop(columns=[0])
x.insert(0,'bias',1)
x.head()

,bias,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,745,746,747,748,749,750,751,752,753,754,755,756,757,758,759,760,761,762,763,764,765,766,767,768,769,770,771,772,773,774,775,776,777,778,779,780,781,782,783,784
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,255,255,255,255,255,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
x=np.array(x)

In [ ]:
y=data[0]
y.shape

(3599,)

In [ ]:
y=np.array(y)
y=np.reshape(y,(-1,1))

In [ ]:
x.shape, y.shape

((3599, 785), (3599, 1))

In [ ]:
b = np.unique(y).size  #finding all unique classes in the output
print(b)


4


In [ ]:
classes = np.unique(y).size  #finding all unique classes in the output
print(classes)

4


In [ ]:
params = x.shape[1]               #Finding the number of features
all_theta = np.zeros((classes, params)) # here 4 denotes number of labels (0 to 3)
all_theta.shape

(4, 785)

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=2)

x_train.shape, x_test.shape, y_train.shape, y_test.shape

((2699, 785), (900, 785), (2699, 1), (900, 1))

In [ ]:
def sigmoid(z):    #returns values between zero to one
    return 1 / (1 + np.exp(-z))

In [ ]:
sigmoid(-10)

4.5397868702434395e-05

In [ ]:
def computeCost(theta, x, y):
    theta = np.matrix(theta)
    x = np.matrix(x)
    y = np.matrix(y)

    #(-y*log(g(z))) --> when expected output (y)=1
    first = np.multiply(-y, np.log(sigmoid(x * theta.T)))

    #((1-y)*(1-log(g(z)))) --> when expected output (y)=0
    second = np.multiply((1 - y), np.log(1 - sigmoid(x * theta.T)))

    cost = np.sum(first - second)
    cost = cost/(2*len(x))

    return cost

In [ ]:
def gradient(theta, x, y):

    theta = np.matrix(theta)
    x = np.matrix(x)
    y = np.matrix(y)
    learning_rate = 1

    error = sigmoid(x * theta.T) - y
    grad = learning_rate*((x.T * error) / len(x))

    return grad

In [ ]:
from scipy.optimize import minimize
def one_vs_all(x, y, classes, new_theta):
        rows = x.shape[0]

        for i in range(0, classes):
            theta = new_theta[i,:]

            y_i = np.array([1 if label == i else 0 for label in y])
            y_i = np.reshape(y_i, (rows, 1))

        # minimize the objective function  --> Gradient descent
            fmin = minimize(fun=computeCost, x0=theta, args=(x, y_i), method='TNC', jac=gradient)
            all_theta[i,:] = fmin.x

            print(fmin.fun)
        return all_theta


In [ ]:
final_theta = one_vs_all(x_train,y_train, classes, all_theta)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: RuntimeWarning: divide by zero encountered in log
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: RuntimeWarning: invalid value encountered in multiply
  # Remove the CWD from sys.path while we load stuff.


0.012821566335776193
0.06081347133361224
0.027065551573512207
0.032967429204379096


In [ ]:
print('optimized_theta values = {}'.format(final_theta))

optimized_theta values = [[-0.00277556  0.          0.         ... -0.00062547 -0.00053309
  -0.00053309]
 [ 0.00291602  0.          0.         ... -0.00083366  0.00032797
   0.00032797]
 [-0.01889743  0.          0.         ...  0.01314427  0.00026361
   0.00026361]
 [-0.01602643  0.          0.         ... -0.01180887 -0.00904963
  -0.00904963]]


In [ ]:
def predict_all(x, theta):

    # convert to matrices
    x = np.matrix(x)
    theta = np.matrix(theta)

    # run the model on each training instance
    h = sigmoid(x * theta.T)

    # return the column index that has the maximum confidence
    h_argmax = np.argmax(h, axis=1)

    return h_argmax, h

In [ ]:
y_pred, h = predict_all(x_train, final_theta)
#print(y_pred)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_pred,y_train)
print('accuracy = {} % '.format (round(accuracy,2)*100))

accuracy = 98.0 % 


In [ ]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(x_pred,x_train)
print('accuracy = {} % '.format (round(accuracy,2)*100))

NameError: ignored

In [ ]:
t_y_pred, h = predict_all(x_test, final_theta)

In [ ]:
accuracy_test = accuracy_score(t_y_pred,y_test)
print('accuracy = {} % '.format (round(accuracy_test,2)*100))

accuracy = 76.0 % 


In [ ]:
from PIL import Image
from PIL import ImageOps
from IPython.display import display

Im= Image.open(path+'0.png')
display(Im)

In [ ]:
Im= Image.open(path+'3.png')
display(Im)

In [ ]:
Im=Im.convert('L')    # convert to grayScale
Im = ImageOps.invert(Im) #invert the Image
display(Im)

In [ ]:
Im = np.array(Im)
Im.shape

(784,)

In [ ]:
Im = np.ravel(Im)
Im.shape

(784,)

In [ ]:
Im= Im.reshape(1,-1)
input_x = np.insert(Im, 0, 1, axis=1)

In [ ]:
input_x.shape

(1, 785)

In [ ]:
pred, h = predict_all(input_x, final_theta)

print ("confidence matrix={} ".format(h))

print('predicted class is : {}'.format(pred.item()))

print ("confidence={} %".format(((h[0,pred]) * 100).item()))

confidence matrix=[[4.44947475e-11 9.61389450e-05 1.46986823e-03 9.99993152e-01]] 
predicted class is : 3
confidence=99.99931516398716 %
